In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\codes\\mlops\\Kidney-Disease-Classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path : Path
    training_path : Path
    params_image_size : list
    all_params : dict
    params_batch_size : int
    

In [4]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml , create_direcories 

In [5]:
class ConfigurationManager:
    def __init__(self, CONFIG_FILE_PATH = CONFIG_FILE_PATH, PARAMS_FILE_PATH = PARAMS_FILE_PATH):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_direcories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            model_path = Path(self.config.training.trained_model_path),
            training_path = Path("artifacts/data_ingestion/kidney-ct-scan-image"),
            params_image_size = self.params.IMAGE_SIZE,
            all_params = self.params,
            params_batch_size = self.params.BATCH_SIZE,
        )
        return eval_config

In [6]:
import tensorflow as tf
from src.cnnClassifier.utils.utils import save_json

In [14]:
class Evaluation:
    def __init__(self, config : EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        data_generator = dict(
            rescale = 1./255,
            validation_split = 0.2
        )
        
        data_flow = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear'
        )
        valid_datagenerator  = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator # type: ignore
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory= self.config.training_path,
            subset = "validation",
            shuffle = True,
            **data_flow # type: ignore
        )
        
    @staticmethod
    def load_model(path : Path)->tf.keras.Model:
            return tf.keras.models.load_model(path) #type: ignore
        
    def evaluation(self):
            self.model = self.load_model(self.config.model_path)
            self._valid_generator()
            self.score = self.model.evaluate(self.valid_generator)
            
    def save_score(self):
            scores  = {"loss ": self.score[0], "accuracy" : self.score[1]}
            save_json(path = Path("scores.json"), data= scores)

In [15]:
import sys
from src.cnnClassifier.exception.exception  import  customexception

In [17]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation._valid_generator()
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise customexception(e, sys)

[2025-01-02 20:41:32,301: INFO: utils: yaml file : config\config.yaml read successfully]
[2025-01-02 20:41:32,309: INFO: utils: yaml file : params.yaml read successfully]
[2025-01-02 20:41:32,312: INFO: utils: directory : artifacts created successfully]
Found 93 images belonging to 2 classes.
Found 93 images belonging to 2 classes.
10/10 [==============================] - 16s 1s/step - loss: 0.5431 - accuracy: 0.8602
[2025-01-02 20:41:56,957: INFO: utils: json file saved at: scores.json]
